# Chess Engine with PyTorch

## Imports

In [1]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import random_split, DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from auxiliary_func import check_memory

2025-11-14 23:44:51.934034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-14 23:44:51.961091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-14 23:44:51.961140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-14 23:44:51.974613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-14 23:44:52.939375: W tensorflow/compiler/tf

# Data preprocessing

## Load data

In [2]:
# Calcute memory distribution so that loading pgns is 10% of processed data, 1.5 gb leftover

total_mem = check_memory()
pgn_memory_mark = total_mem*3/7
print(total_mem)
print(pgn_memory_mark)

7.888668060302734
3.3808577401297435


In [ ]:
from auxiliary_func import load_dataset, encode_moves
files = [file for file in os.listdir("../../data") if file.endswith(".pgn")]
# Sort by file size (ascending)
files_sorted = sorted(files, key=lambda f: os.path.getsize(os.path.join("../../data", f)))

LIMIT_OF_FILES = min(len(files_sorted), 79)
files_parsed = 0
games_parsed = 0
stop = False

X, y = load_dataset(files_sorted, pgn_memory_mark=pgn_memory_mark, file_limit=LIMIT_OF_FILES)




 41%|████      | 32/79 [09:27<13:53, 17.74s/it]

Completed sampling 32 files with 4.894161224365234 remaining


In [4]:
X, y = np.array(X, dtype=np.float32), np.array(y)

y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [5]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [6]:
print(f"GAMES PARSED: {games_parsed}")
print(f"FILES PARSED: {files_parsed}")
print(f"MOVES RECORDED: {len(y)}")
available_gb = check_memory()
print(f"Available Memory: {available_gb}")

GAMES PARSED: 0
FILES PARSED: 0
MOVES RECORDED: 1105294
Available Memory: 3.5345306396484375


# Preliminary actions

In [7]:
from dataset import ChessDataset
from model import ChessModel

In [8]:
# Create Dataset
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Compute split sizes
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Then create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cuda


# Data Collection

In [9]:
# Get current time in a readable format
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")

# Create a unique log directory
log_dir = f"../../runs/400Epochs_default_model_random_data_experiment_i{len(y)}_{current_time}"

# Create the SummaryWriter
writer = SummaryWriter(log_dir=log_dir)

# Training

In [10]:
num_epochs = 200
for epoch in range(num_epochs):
    start_time = time.time()

    # Training
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()


    # Stats
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60

    if epoch % 40 == 0:
        # Save the model
        torch.save(model.state_dict(), f"../../models/checkpoints/TORCH_{epoch}EPOCHS_random_data.pth")  
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, Time: {minutes}m{seconds}s')

    writer.add_scalar("Loss/train", avg_train_loss, epoch + 1)
    writer.add_scalar("Loss/validation", avg_val_loss, epoch + 1)

writer.close()

100%|██████████| 15544/15544 [00:59<00:00, 262.60it/s]


Epoch 1/200, Loss: 4.2366, Time: 1m0s


100%|██████████| 15544/15544 [00:58<00:00, 264.96it/s]


Epoch 2/200, Loss: 3.1463, Time: 1m0s


100%|██████████| 15544/15544 [00:58<00:00, 266.22it/s]


...

Epoch 198/200, Loss: 0.3835, Time: 1m0s


100%|██████████| 15544/15544 [00:58<00:00, 265.34it/s]


Epoch 199/200, Loss: 0.3825, Time: 1m0s


100%|██████████| 15544/15544 [00:58<00:00, 263.92it/s]


Epoch 200/200, Loss: 0.3811, Time: 1m0s


# Save the model and mapping

In [11]:
# Save the model
torch.save(model.state_dict(), "../../models/400EPOCHS_random_data.pth")

In [12]:
import pickle

with open("../../models/random_data_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)